# COVID-19 Data Visualization and Exploratory Data Analysis

## Data Source
The following is a portfolio project in which the Our World in Data Covid-19 dataset is explored.  A copy of the dataset can be found at [this site](https://github.com/owid/covid-19-data/tree/master/public/data).

The data comes from the COVID-19 Data Repository by the Center of Systems Science and Engineering (CSSE) at Johns Hopkins University.  The dataset contains data regarding confirmed cases and deaths, hospitalizations, testing for COVID-19, vaccinations against COVID-19, and other variables.

## Data Ingestion
Ingested the data into a SQL server instance.  The data should have two tables: CovidDeaths and CovidVaccinations.  Alternatively, use the code below to create the database in your SQL server instance.

In [ ]:
/* put SQL database backup code here */